In [1]:
import pandas as pd 
import numpy as np
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
import re
data = pd.read_csv('Tesla.csv')
cleaned_data = pd.DataFrame()
cleaned_data['original_tweet'] = data['tweet']
cleaned_data['cleaned_tweet'] = ""
cleaned_data['sentiment'] = ""
cleaned_data['compound_score'] = 0.0
cleaned_data['positive_score'] = 0.0
cleaned_data['negative_score'] = 0.0
cleaned_data['neutral_score'] = 0.0
#Preprocess our data to be a valid input for VADER
# --> VADER can handle emojis, punctuation, capitalization, slang, and negations ... so don't need to remove these items

cleaned_tweets = []
for tweet in data["tweet"]:
    tweet = str(tweet)

    #remove urls
    tweet = re.sub(r'http\S+|www\S+|https\S+', '', tweet)

    #remove @'s
    tweet = re.sub(r'@\w+', '', tweet)

    #remove #'s
    tweet = re.sub(r'#', '', tweet)

    #remove special chars
    tweet = re.sub(r'[^\w\s]', '', tweet)

    #remove nums
    tweet = re.sub(r'\d+', '', tweet)

    #remove extra spaces
    tweet = tweet.strip()

    #add cleaned tweet to the arr
    cleaned_tweets.append(tweet)

cleaned_data["cleaned_tweet"] = cleaned_tweets
#create a VADER setniment analyzer:
sentiment_analyzer = SentimentIntensityAnalyzer()

# Get the sentiment/polarity score of each tweet:
for i in range(len(cleaned_data)):

    curr_tweet = cleaned_data["cleaned_tweet"][i]
    
    # curr_score is a dictionary that stores a positive, negative, neutral, and compound score for an individual tweet
    curr_score = sentiment_analyzer.polarity_scores(curr_tweet)

    cleaned_data.loc[i, "compound_score"] = curr_score["compound"]
    cleaned_data.loc[i, "positive_score"] = curr_score["pos"]
    cleaned_data.loc[i, "negative_score"] = curr_score["neg"]
    cleaned_data.loc[i, "neutral_score"] = curr_score["neu"]

    # Determine sentiment category
    if curr_score['compound'] >= 0.05:
        cleaned_data.loc[i, 'sentiment'] = 'Positive'
    elif curr_score['compound'] <= -0.05:
        cleaned_data.loc[i, 'sentiment'] = 'Negative'
    else:
        cleaned_data.loc[i, 'sentiment'] = 'Neutral'

# ////////////////////////////

import pandas as pd
from scipy.stats import spearmanr

stock_data['Date'] = pd.to_datetime(stock_data['Date']).dt.date
daily_sentiment['date'] = pd.to_datetime(daily_sentiment['date']).dt.date

combined_data = pd.merge(stock_data, daily_sentiment, left_on='Date', right_on='date', how='inner')

combined_data.drop(columns='date', inplace=True)

stock_prices = combined_data['Close']
compound_scores = combined_data['compound_score']

rho, p_value = spearmanr(stock_prices, compound_scores)
print("Spearman's correlation (rho) between Close price and compound_score:", rho)
print("p-value:", p_value)

sentiment_columns = ['compound_score', 'positive_score', 'negative_score', 'neutral_score']

for sentiment_col in sentiment_columns:
    rho, p_value = spearmanr(combined_data['Close'], combined_data[sentiment_col])
    print(f"Spearman's correlation between Close and {sentiment_col}: rho={rho:.4f}, p={p_value:.4f}")

NameError: name 'stock_data' is not defined